In [1]:
import numpy as np 
import pandas as pd 


In [2]:
import tensorflow as tf
from tensorflow import keras


In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


From https://www.tensorflow.org/api_docs/python/tf/nn/conv2d

Given an input tensor of shape batch_shape + [in_height, in_width, in_channels] and a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels], this op performs the following:

Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].

Extracts image patches from the input tensor to form a virtual tensor of shape [batch, out_height, out_width, filter_height * filter_width * in_channels].

For each patch, right-multiplies the filter matrix and the image patch vector.

In [4]:
class spectralConv2D(keras.layers.Layer):
    
    def __init__(self, filters = 32,kernel_size=(3,3),strides=(1, 1), padding='VALID',activation=tf.nn.relu,input_shape=None):
        
        # Initialization
        super(spectralConv2D, self).__init__()
        assert len(kernel_size) > 1 , "Please input the Kernel Size as a 2D tuple"
        self.strides = strides 
        self.padding = padding
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = activation
        
    def build(self, input_shape):
        # Initialize Filters 
        # Assuming Input_shape is channels_last 
        kernel_shape = [self.kernel_size[0],self.kernel_size[1],input_shape[3],self.filters]
        
        self.kernel_real = self.add_weight( shape=kernel_shape,
                                            initializer="glorot_uniform",
                                            trainable=True)
        self.kernel_imag = self.add_weight( shape=kernel_shape,
                                            initializer="glorot_uniform",
                                            trainable=True)
        
        self.bias = self.add_weight(shape=(self.filters,), initializer="zeros", trainable=True)

    def call(self, inputs):
        print(self.kernel_real)
        kernel = tf.complex(self.kernel_real,self.kernel_imag)
        
        spatial_kernel = tf.signal.ifft2d(kernel)
                
        spatial_kernel=tf.abs(spatial_kernel)
        
        convolution_output = tf.nn.convolution(
                            inputs,
                            spatial_kernel,
                            strides=list(self.strides),
                            padding=self.padding
                        )
        
        convolution_output = tf.nn.bias_add(convolution_output, self.bias)
        
        if self.activation is not None:
            convolution_output= self.activation(convolution_output)
            
        return convolution_output
    

In [5]:
# CIFAR10 Dataset
from modules.utils import load_data
X_train, y_train = load_data(mode='train')
num_training = 49000
num_validation = 1000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

# Preprocessing: subtract the mean value across every dimension for training data, and reshape it to be RGB size
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
X_val = X_val.astype(np.float32) - mean_image

X_train = X_train.reshape(-1,3,32,32).transpose(0,2,3,1) / 255
X_val = X_val.reshape(-1,3,32,32).transpose(0,2,3,1) / 255

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

./data/cifar-10-python.tar.gz already exists. Begin extracting...
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,ReLU,GlobalAveragePooling2D,Softmax,Flatten,Dense #, , AveragePooling2D, MaxPooling2D,,
from tensorflow.keras import Model,Input

In [7]:
model = Sequential()
model.add(spectralConv2D(32, (3,3),input_shape=X_train.shape[1:]))
model.add(spectralConv2D(64, (3,3)))
model.add(spectralConv2D(128, (3,3)))
model.add(spectralConv2D(94, (3,3)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10,activation="softmax"))

In [8]:
model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [9]:
model.fit(x=X_train, y=y_train,
          batch_size=128,
          epochs=5, 
          validation_data=(X_val, y_val)
              )

Epoch 1/5
<tf.Variable 'spectral_conv2d/Variable:0' shape=(3, 3, 3, 32) dtype=float32>
<tf.Variable 'spectral_conv2d_1/Variable:0' shape=(3, 3, 32, 64) dtype=float32>
<tf.Variable 'spectral_conv2d_2/Variable:0' shape=(3, 3, 64, 128) dtype=float32>
<tf.Variable 'spectral_conv2d_3/Variable:0' shape=(3, 3, 128, 94) dtype=float32>
<tf.Variable 'spectral_conv2d/Variable:0' shape=(3, 3, 3, 32) dtype=float32>
<tf.Variable 'spectral_conv2d_1/Variable:0' shape=(3, 3, 32, 64) dtype=float32>
<tf.Variable 'spectral_conv2d_2/Variable:0' shape=(3, 3, 64, 128) dtype=float32>
<tf.Variable 'spectral_conv2d_3/Variable:0' shape=(3, 3, 128, 94) dtype=float32>
<tf.Variable 'spectral_conv2d/Variable:0' shape=(3, 3, 3, 32) dtype=float32>
<tf.Variable 'spectral_conv2d_1/Variable:0' shape=(3, 3, 32, 64) dtype=float32>
<tf.Variable 'spectral_conv2d_2/Variable:0' shape=(3, 3, 64, 128) dtype=float32>
<tf.Variable 'spectral_conv2d_3/Variable:0' shape=(3, 3, 128, 94) dtype=float32>
383/383 [========================

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spectral_conv2d (spectralCon (None, 30, 30, 32)        1760      
_________________________________________________________________
spectral_conv2d_1 (spectralC (None, 28, 28, 64)        36928     
_________________________________________________________________
spectral_conv2d_2 (spectralC (None, 26, 26, 128)       147584    
_________________________________________________________________
spectral_conv2d_3 (spectralC (None, 24, 24, 94)        216670    
_________________________________________________________________
flatten (Flatten)            (None, 54144)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               6930560   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1